In [1]:
def read_311_data(datafile):
    import pandas as pd
    import numpy as np

    #fix_zip: clean the format of zip code and only retain zip between 10000 and 19999
    def fix_zip(input_zip):
        try:
            input_zip = int(float(input_zip))
        except:
            try:
                input_zip = int(input_zip.split('-')[0])
            except:
                return np.NaN
        if input_zip < 10000 or input_zip > 19999:
            return np.NaN
        return str(input_zip)
    
    #Read the file and apply fix_zip
    df = pd.read_csv(datafile,index_col='Unique Key')
    df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    df = df.dropna(how='any')
    
    #get rid of unspecified boroughs
    df = df[df['Borough'] != 'Unspecified']
    
    #Convert times to datetime and create a processing time column
    import datetime
    df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['processing_time'] =  df['Closed Date'] - df['Created Date']
    
    #Finally, get rid of negative processing times and return the final data frame
    df = df[df['processing_time']>=datetime.timedelta(0,0,0)]
    
    return df

# Deal with NaN value

In [ ]:
df = read_311_data(datafile)
df = df.dropna() # drop nan value in all columns
df = df[df['Incident Zip'].notnull()] # only drop nan value in Incident Zip column
nypd_complaints_total = df[df['Agency']=='NYPD']['Borough'].count() 
# just give back the number of rows of NYPD, .count() only works on Series object

# Deal with datetime value

In [ ]:
import datetime
import numpy as np
import pandas as pd

# 1. strptime: string to pandas._libs.tslibs.timestamps.Timestamp object
df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p')) # from string to datetime object
df[df['processing_time']<datetime.timedelta(0,0,0)]
# processing_time: pandas._libs.tslibs.timedeltas.Timedelta
df['processing_time'] = df['Closed Date'] - df['Created Date']
df['float_time'] = df['processing_time'].apply(lambda x:x/np.timedelta64(1, 'D')) # timedelta 1 day, convert into float of how many days
df['hour of day'] = df['Created Date'].apply(lambda x:x.hour) # retrive hour of the day
df['yyyymm'] = df['Created Date'].apply(lambda x:datetime.datetime.strftime(x,'%Y%m')) # from datetime object to string with specified format

# 2. pd.to_datetime: string to pandas._libs.tslibs.timestamps.Timestamp object
# Converts a scalar, array-like, Series or DataFrame/dict-like to a pandas datetime object.
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"],infer_datetime_format=True)
df['day_of_week'] = df['tpep_pickup_datetime'].apply(lambda x: x.isoweekday())
df['pickup_hour'] = df['tpep_pickup_datetime'].apply(lambda x: x.hour)
# trip_duration: pandas._libs.tslibs.timedeltas.Timedelta
df['trip_duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['trip_duration'] = df['trip_duration'].apply(lambda x: x/np.timedelta64(1,'s')) # timedelta 1 second, convert into float of how many seconds